In [1]:
%cd /content/drive/My Drive/simpleT

/content/drive/My Drive/simpleT


In [2]:
#!git clone https://github.com/ThilinaRajapakse/simpletransformers.git

Cloning into 'simpletransformers'...
remote: Enumerating objects: 4653, done.
remote: Total 4653 (delta 0), reused 0 (delta 0), pack-reused 4653
Receiving objects: 100% (4653/4653), 18.24 MiB | 11.28 MiB/s, done.
Resolving deltas: 100% (2833/2833), done.
Checking out files: 100% (426/426), done.


In [2]:
!pip install simpletransformers

In [2]:
import pandas as pd 
#train_data = pd.read_csv("NL_SQL_train.csv") 
test_data = pd.read_csv("NL_SQL_test.csv") 
#train_data.head()
test_data.head()

,prefix,Question,SQL
0,translate English to SQL,what flights are available tomorrow from denve...,SELECT DISTINCT flight_1.flight_id FROM fligh...
1,translate English to SQL,show me the afternoon flights from washington ...,SELECT DISTINCT flight_1.flight_id FROM fligh...
2,translate English to SQL,flights from phoenix to milwaukee,SELECT DISTINCT flight_1.flight_id FROM fligh...
3,translate English to SQL,i'd like to fly from philadelphia to san franc...,SELECT DISTINCT flight_1.flight_id FROM fligh...
4,translate English to SQL,i'm starting from denver,SELECT DISTINCT flight_1.flight_id FROM fligh...


In [3]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_data = pd.read_csv("NL_SQL_train.csv") 
test_data = pd.read_csv("NL_SQL_test.csv") 
train_data.rename(columns = {'Question':'input_text',"SQL":"target_text"}, inplace = True) 
test_data.rename(columns = {'Question':'input_text',"SQL":"target_text"}, inplace = True) 


train_df = train_data
train_df = pd.DataFrame(train_data, columns=["prefix", "input_text", "target_text"])

eval_df=test_data
eval_df = pd.DataFrame(test_data, columns=["prefix", "input_text", "target_text"])

eval_df = train_df.copy()

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 10,
    "train_batch_size": 2,
    "num_train_epochs": 2,
}

# Create T5 Model
model = T5Model("t5-base", args=model_args,use_cuda=False)

# Train T5 Model on new task
model.train_model(train_df)

# Evaluate T5 Model on new task
results = model.eval_model(eval_df)

# Predict with trained T5 model
print(model.predict(["translate English to SQL: flights from phoenix to milwaukee"]))
print(model.predict(["translate English to SQL: i'm starting from denver"]))

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_106079


/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Running loss: 4.120878

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

INFO:simpletransformers.t5.t5_model: Training of t5-base model complete. Saved to outputs/.
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_106079


INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.11459356559391477}



['SELECT DISTINCT flight_1.flight_1.flight_1.f']



['SELECT DISTINCT flight_1.f.f1.f1.f1.f']


In [8]:
print(model.predict(["translate English to SQL: flights from phoenix to milwaukee"]))
print(model.predict(["translate English to SQL: i'm starting from denver"]))
print(model.predict(["translate English to SQL: denver to atlanta"]))
print(model.predict(["translate English to SQL: show me the flights from baltimore to atlanta"]))



['SELECT DISTINCT flight_1.flight_1.flight_1.f']



['SELECT DISTINCT flight_1.f.f1.f1.f1.f']



['SELECT DISTINCT flight_1.flight_1.flight_1.f']



['SELECT DISTINCT flight_1.flight_1.flight_1.f']


In [6]:
checkpoint_path = "cp-SimpleTModel.ckpt"
#model.saved_model.save(checkpoint_path)
saved_model_path = model.export(
    '/content/drive/My Drive/simpleT',
    checkpoint_step=-1,  # use most recent
    beam_size=1,  # no beam search
    temperature=1.0,  # sample according to predicted distribution
)
print("Model saved to:", saved_model_path)

AttributeError: ignored

,prefix,input_text,target_text
0,translate English to SQL,i want to go from boston to washington on a sa...,SELECT DISTINCT flight.flight_id FROM flight ...
1,translate English to SQL,i need a flight from cleveland to dallas that ...,SELECT DISTINCT flight.flight_id FROM flight ...
2,translate English to SQL,get fares from washington to boston,SELECT DISTINCT fare.fare_id FROM fare WHERE ...
3,translate English to SQL,get saturday fares from washington to boston,SELECT DISTINCT fare.fare_id FROM fare WHERE ...
4,translate English to SQL,get fares from washington to montreal,SELECT DISTINCT fare.fare_id FROM fare WHERE ...
